# Workshop 2: Image Segmentation

In this workshop we will how to implement a segmentation model based in UNet and ResNet with [fastai](https://https://www.fast.ai/). The structure of the workshop will be as follows:


1.   Load and explore data
2.   Train the model
3.   Evaluate results

In [ ]:
from fastai.vision.all import *
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Download dataset
path = untar_data(URLs.CAMVID_TINY)
path.ls()

In [ ]:
# Number of images
len((path / "images").ls())

In [ ]:
# Number of masks
len((path / "labels").ls())

In [ ]:
(path / "images").ls()[0], (path / "labels").ls()[0]

In [ ]:
# Show image and size
img_name = Path("Seq05VD_f00090.png")
img = PILImage.create(path / "images" / img_name)
print(img.shape)
img.show()

In [ ]:
# Show mask and size
mask_name = img_name.stem + "_P" + img_name.suffix
mask = PILMask.create(path / "labels" / mask_name)
print(mask.shape)
mask.show()

In [ ]:
# Show labels names
codes = np.loadtxt(path/'codes.txt', dtype=str)
len(codes), codes

In [ ]:
# Show mask matrix
value_list = list(mask.getdata())
print(len(value_list))
value_list[0], value_list[150], value_list[1238], value_list[5000], value_list[12000]

In [ ]:
min(value_list), max(value_list)

# 1. Load and explore data

In [ ]:
# Function to get label path from image name
def label_func(fn): return path/"labels"/f"{fn.stem}_P{fn.suffix}"

In [ ]:
# Get list of image paths
fnames = get_image_files(path/"images")
fnames[0]

In [ ]:
# Create dataloader
dls = SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames=fnames, label_func=label_func, codes=codes
)

In [ ]:
# Show batch
dls.show_batch(max_n=6)

# 2. Train the model

In [ ]:
# Initialize model
learn = unet_learner(dls, resnet34)

In [ ]:
# Train model
learn.fine_tune(6)

# 3. Show results

In [ ]:
# Show results
learn.show_results(max_n=6, figsize=(7,8))

In [ ]:
# Show worst cases
interp = SegmentationInterpretation.from_learner(learn)
interp.plot_top_losses(k=3)

# Exercise 1: Improve the model using all the techniques seen in the classification notebook. Use fastai documentation if needed.

In [ ]:
# Create dataloader with data augmentation
dls = SegmentationDataLoaders.from_label_func(
    path, bs=8, fnames=fnames, label_func=label_func, codes=codes,
    batch_tfms=aug_transforms(size=224)
)

In [ ]:
# Show batch
dls.show_batch(max_n=6)

In [ ]:
# Initialize learner with DiceMulti as metric
learn = unet_learner(dls, resnet34, metrics=[DiceMulti()])

In [ ]:
# Find lr
learn.lr_find()

In [ ]:
# Train the model
learn.fine_tune(5, 2e-4)

In [ ]:
# Show results

In [ ]:
# Do anything you consider to improve the model